# Assignment 2

In [2]:
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [36]:
# Useful functions 
# ================
def dist(Pos, X, Y, Z):
    x=Pos[0,:]
    y=Pos[1,:]
    z=Pos[2,:]
    R=[]
    for i in range(0, np.shape(Pos)[1]):
        r = np.sqrt((X-x[i])**2 + (Y-y[i])**2 + (Z-z[i])**2)
        R.append(r)
    return np.array(R)

def dist_v(Pos, X, Y, Z):
    x=Pos[0,:]
    y=Pos[1,:]
    z=Pos[2,:]
#     a, b, c = np.shape(X)
#     R = np.zeros(a, b, c)
    R=[]
    for i in range(0, np.shape(Pos)[1]):
        r = np.array([X-x[i], Y-y[i], Z-z[i]])
        R.append(r)
    return np.array(R)

## Question 5

### (a)

We have a loop of radius $0.1m$ lying in the xy-plane with current of $I$ flowing.

To calculate the magnetic field produced by this loop we use the Biot-Savart Law:

$$\vec{B}(\vec{r}) = \frac{\mu_0 I}{4 \pi} \int \frac{\text{d}l \times \hat{r'}}{r' ^2}$$

$$\text{where} \left\{ \begin{array}{ll}
            \vec{r}  & \text{is the position vector of the point of evaluation} \\
            \vec{r'} & \text{is the displacement vector from a point in the current loop to the point of evaluation} \\
            l        & \text{is the length of the current loop} \\
            \end{array} \right.$$      
            
We will now create a meshgrid for the space of integration.

In [3]:
# Dimesions and resolution for the meshgrid 
r_max = 2
r_min = -2
res = 10**(-1)

x = np.arange(r_min, r_max, res)
y = np.arange(r_min, r_max, res)
z = np.arange(r_min, r_max, res)

GX, GY, GZ = np.meshgrid(x, y, z)

Now we create an array of for the position and length of the current loop.

In [29]:
theta = np.linspace(0, 2*np.pi, int(1/(res*2)))

R = np.array([np.cos(theta), np.sin(theta), np.zeros(len(theta))])  # Position of loop segments  
L = np.array([-np.sin(theta), np.cos(theta), np.zeros(len(theta))]) # Lengths of loop segments 

In [37]:
Dist = dist(R, GX, GY, GZ)
Dist_V = dist_v(R, GX, GY, GZ)

In [40]:
np.shape(R)

(3, 5)

In [49]:
np.shape(Dist_V)

(5, 3, 40, 40, 40)

In [50]:
np.shape(Dist)

(5, 40, 40, 40)